# 06 - Dashboard Usage Guide

Learn how to use the Flask-based real-time monitoring dashboard.

## Table of Contents
1. [Overview](#1.-Overview)
2. [Starting the Dashboard](#2.-Starting-the-Dashboard)
3. [Dashboard Features](#3.-Dashboard-Features)
4. [Loading Models and Data](#4.-Loading-Models-and-Data)
5. [Real-time Predictions](#5.-Real-time-Predictions)
6. [Visualization Features](#6.-Visualization-Features)
7. [API Endpoints](#7.-API-Endpoints)
8. [Configuration Options](#8.-Configuration-Options)

---

## 1. Overview

The G-code Fingerprinting Dashboard provides a real-time web interface for:

- **Live Predictions**: Stream sensor data and see predictions in real-time
- **Confusion Matrices**: Track prediction accuracy across classes
- **Sensor Visualization**: Heatmaps and time-series plots
- **Model Management**: Load different checkpoints
- **Data Export**: Download predictions as CSV

### Dashboard Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                   Flask Dashboard (port 5000)                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │    Model     │  │   Sensor     │  │  Prediction  │          │
│  │   Loader     │  │   Stream     │  │   Display    │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
│                                                                  │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │  Confusion   │  │   Sensor     │  │   Export     │          │
│  │   Matrix     │  │   Heatmap    │  │   Tools      │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# Environment Setup
# ============================================================

import sys
from pathlib import Path
import json
import time

import numpy as np
import requests

# Project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

# Dashboard Configuration
DASHBOARD_URL = 'http://localhost:5000'

print("="*60)
print("G-CODE FINGERPRINTING DASHBOARD")
print("="*60)
print(f"Dashboard URL: {DASHBOARD_URL}")
print(f"Project Root: {project_root}")

## 2. Starting the Dashboard

Start the Flask dashboard in a separate terminal:

```bash
# Recommended: Run with PYTHONPATH set
PYTHONPATH=src .venv/bin/python flask_dashboard.py

# Alternative: Direct execution
cd /path/to/gcode_fingerprinting
.venv/bin/python flask_dashboard.py
```

The dashboard will:
1. Start on http://localhost:5000
2. Initialize SocketIO for real-time updates
3. Optionally connect to Redis for caching

In [ ]:
# Check if dashboard is running
def check_dashboard_status():
    """Check if the dashboard is running."""
    try:
        response = requests.get(f'{DASHBOARD_URL}/', timeout=5)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False
    except Exception:
        return False

is_running = check_dashboard_status()

if is_running:
    print("\n✓ Dashboard is running!")
    print(f"  Open in browser: {DASHBOARD_URL}")
else:
    print("\n✗ Dashboard is not running!")
    print("  Start with: PYTHONPATH=src .venv/bin/python flask_dashboard.py")

## 3. Dashboard Features

### Main Features

| Feature | Description |
|---------|-------------|
| **Model Selector** | Choose from available checkpoints |
| **CSV Upload** | Load sensor data from CSV files |
| **Live Predictions** | Real-time G-code prediction stream |
| **Confusion Matrix** | Per-head accuracy visualization |
| **Sensor Heatmap** | Visualize sensor activation patterns |
| **3D Position Plot** | Track predicted XYZ movements |
| **Dark Mode** | Toggle dark/light theme |
| **CSV Export** | Download prediction results |

### Operation Types

The dashboard tracks 9 CNC operation classes:

```python
OPERATION_TYPES = [
    "adaptive",        # 0
    "adaptive150025",  # 1
    "face",            # 2
    "face150025",      # 3
    "pocket",          # 4
    "pocket150025",    # 5
    "damageadaptive",  # 6
    "damageface",      # 7
    "damagepocket",    # 8
]
```

## 4. Loading Models and Data

### Available Models

In [ ]:
# Get available models from dashboard API
def get_available_models():
    """Fetch list of available models from dashboard."""
    try:
        response = requests.get(f'{DASHBOARD_URL}/api/models', timeout=5)
        if response.status_code == 200:
            return response.json()
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

models = get_available_models()
if models:
    print("\nAvailable Models:")
    print("-" * 50)
    for model in models.get('models', [])[:5]:
        print(f"  • {model}")
else:
    print("\nCould not fetch models (dashboard may not be running)")

In [ ]:
# Get available CSV files
def get_available_csv_files():
    """Fetch list of available CSV data files."""
    try:
        response = requests.get(f'{DASHBOARD_URL}/api/csv_files', timeout=5)
        if response.status_code == 200:
            return response.json()
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

csv_files = get_available_csv_files()
if csv_files:
    print("\nAvailable CSV Files:")
    print("-" * 50)
    for csv_file in csv_files.get('files', [])[:5]:
        print(f"  • {csv_file}")
else:
    print("\nCould not fetch CSV files")

In [ ]:
# Load a model programmatically
def load_model(checkpoint_name):
    """Load a specific model checkpoint."""
    payload = {'checkpoint': checkpoint_name}
    try:
        response = requests.post(
            f'{DASHBOARD_URL}/api/load_model',
            json=payload,
            timeout=30
        )
        return response.json()
    except Exception as e:
        return {'error': str(e)}

# Example usage (uncomment to test)
# result = load_model('checkpoint_best.pt')
# print(f"Load result: {result}")

print("\nTo load a model programmatically:")
print("  result = load_model('checkpoint_name.pt')")

## 5. Real-time Predictions

The dashboard uses WebSocket (SocketIO) for real-time streaming.

In [ ]:
# SocketIO client example
try:
    from socketio import Client
    SOCKETIO_AVAILABLE = True
except ImportError:
    SOCKETIO_AVAILABLE = False
    print("python-socketio not installed. Install with: pip install python-socketio")

if SOCKETIO_AVAILABLE:
    print("\nSocketIO Client Example:")
    print("-" * 50)
    print("""
from socketio import Client

sio = Client()

@sio.on('prediction')
def on_prediction(data):
    print(f"Prediction: {data['gcode']}")
    print(f"Confidence: {data['confidence']:.2%}")

@sio.on('confusion_matrix')
def on_confusion(data):
    print(f"Updated confusion matrix: {len(data)} classes")

sio.connect('http://localhost:5000')
sio.wait()  # Wait for events
    """)

In [ ]:
# Simulate prediction stream visualization
import matplotlib.pyplot as plt

# Generate simulated prediction data
np.random.seed(42)
n_steps = 50

# Simulated confidence scores over time
type_conf = 0.85 + 0.1 * np.random.randn(n_steps).cumsum() * 0.01
type_conf = np.clip(type_conf, 0.6, 1.0)

command_conf = 0.75 + 0.1 * np.random.randn(n_steps).cumsum() * 0.01
command_conf = np.clip(command_conf, 0.5, 0.95)

param_type_conf = 0.80 + 0.1 * np.random.randn(n_steps).cumsum() * 0.01
param_type_conf = np.clip(param_type_conf, 0.6, 0.95)

param_value_conf = 0.65 + 0.1 * np.random.randn(n_steps).cumsum() * 0.01
param_value_conf = np.clip(param_value_conf, 0.4, 0.85)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(type_conf, label='Type', linewidth=2)
ax.plot(command_conf, label='Command', linewidth=2)
ax.plot(param_type_conf, label='Param Type', linewidth=2)
ax.plot(param_value_conf, label='Param Value', linewidth=2)

ax.set_xlabel('Time Step')
ax.set_ylabel('Confidence')
ax.set_title('Simulated Real-time Prediction Confidence')
ax.legend(loc='lower right')
ax.set_ylim(0.3, 1.05)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThis visualization simulates the real-time confidence display in the dashboard.")

## 6. Visualization Features

### Sensor Heatmaps

In [ ]:
# Fetch sensor heatmap data
def get_sensor_heatmap():
    """Fetch sensor vs time heatmap data."""
    try:
        response = requests.get(f'{DASHBOARD_URL}/api/heatmap/sensors_vs_time', timeout=10)
        if response.status_code == 200:
            return response.json()
        return None
    except Exception as e:
        return None

# Example heatmap visualization
print("\nSensor Heatmap Visualization:")
print("-" * 50)

# Generate example heatmap
np.random.seed(42)
n_sensors = 20
n_timesteps = 64

# Simulate sensor data with patterns
sensor_data = np.zeros((n_sensors, n_timesteps))
for i in range(n_sensors):
    freq = 0.1 + i * 0.02
    phase = i * 0.5
    sensor_data[i] = np.sin(np.linspace(0, 4*np.pi, n_timesteps) * freq + phase)
    sensor_data[i] += 0.2 * np.random.randn(n_timesteps)

fig, ax = plt.subplots(figsize=(14, 6))

im = ax.imshow(sensor_data, aspect='auto', cmap='viridis')
ax.set_xlabel('Time Step')
ax.set_ylabel('Sensor Channel')
ax.set_title('Sensor Activation Heatmap (Example)')
plt.colorbar(im, label='Activation')

plt.tight_layout()
plt.show()

print("\nThe dashboard displays this heatmap in real-time as data streams in.")

In [ ]:
# Confusion matrix visualization
print("\nConfusion Matrix Example:")
print("-" * 50)

# Generate example confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Simulated predictions for 9 operation classes
np.random.seed(42)
n_samples = 500
n_classes = 9

# Create somewhat realistic predictions (high accuracy with some confusion)
true_labels = np.random.randint(0, n_classes, n_samples)
pred_labels = true_labels.copy()

# Add some errors (15% error rate)
error_mask = np.random.random(n_samples) < 0.15
pred_labels[error_mask] = np.random.randint(0, n_classes, error_mask.sum())

cm = confusion_matrix(true_labels, pred_labels)

# Normalize
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Operation type names
op_names = ['adapt', 'adpt150', 'face', 'face150', 'pocket', 'pkt150', 'dmg_adp', 'dmg_fce', 'dmg_pkt']

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(
    cm_normalized,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=op_names,
    yticklabels=op_names,
    ax=ax
)

ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_title('Operation Type Confusion Matrix (Normalized)')

plt.tight_layout()
plt.show()

accuracy = np.diag(cm).sum() / cm.sum()
print(f"\nOverall Accuracy: {accuracy:.2%}")

## 7. API Endpoints

The dashboard exposes several REST API endpoints.

In [ ]:
# Dashboard API reference
api_endpoints = {
    'GET /': 'Main dashboard page',
    'GET /api/models': 'List available model checkpoints',
    'GET /api/csv_files': 'List available CSV data files',
    'POST /api/load_model': 'Load a specific model checkpoint',
    'POST /api/load_csv': 'Load a CSV file for streaming',
    'GET /api/heatmap/sensors_vs_time': 'Get sensor heatmap data',
    'GET /api/heatmap/sensors_vs_vibration': 'Get vibration correlation heatmap',
}

print("\nDashboard API Endpoints:")
print("="*60)

for endpoint, description in api_endpoints.items():
    method, path = endpoint.split(' ', 1)
    print(f"  {method:6s} {path:40s} {description}")

In [ ]:
# Test all endpoints
def test_dashboard_endpoints():
    """Test all dashboard API endpoints."""
    results = {}
    
    endpoints = [
        ('GET', '/'),
        ('GET', '/api/models'),
        ('GET', '/api/csv_files'),
    ]
    
    for method, path in endpoints:
        try:
            if method == 'GET':
                response = requests.get(f'{DASHBOARD_URL}{path}', timeout=5)
            status = response.status_code
            results[path] = '✓' if status == 200 else f'✗ ({status})'
        except Exception as e:
            results[path] = f'✗ ({str(e)[:20]})'
    
    return results

print("\nEndpoint Health Check:")
print("-" * 40)

results = test_dashboard_endpoints()
for endpoint, status in results.items():
    print(f"  {endpoint:30s} {status}")

## 8. Configuration Options

### Generation Settings

The dashboard supports various generation configurations:

In [ ]:
# Default generation settings
default_settings = {
    'enable_autoregressive': False,  # Disabled by default for performance
    'max_tokens': 15,
    'temperature': 1.0,
    'top_p': 1.0,
    'beam_size': 1,
    'use_beam_search': False,
}

print("\nGeneration Settings:")
print("="*50)

for key, value in default_settings.items():
    print(f"  {key:25s} {value}")

print("\n" + "="*50)
print("Configuration Tips:")
print("-" * 50)
print("""
• enable_autoregressive: Enable for better quality (slower)
• max_tokens: Maximum tokens to generate per sample
• temperature: Higher = more random, Lower = more deterministic
• top_p: Nucleus sampling threshold
• beam_size: Number of beams for beam search
""")

In [ ]:
# Dashboard startup checklist
print("\nDashboard Startup Checklist:")
print("="*50)

checklist = [
    ('Virtual environment activated', '.venv/bin/activate'),
    ('PYTHONPATH set', 'export PYTHONPATH=src'),
    ('Model checkpoint available', 'outputs/*/checkpoint_best.pt'),
    ('Vocabulary file exists', 'data/gcode_vocab_v2.json'),
    ('Port 5000 available', 'lsof -i :5000'),
    ('Redis running (optional)', 'redis-cli ping'),
]

for item, command in checklist:
    print(f"  [ ] {item}")
    print(f"      Check: {command}")

print("\n" + "-"*50)
print("Start command:")
print("  PYTHONPATH=src .venv/bin/python flask_dashboard.py")

## Summary

In this notebook, you learned:

- **Dashboard architecture**: Flask + SocketIO for real-time updates
- **Starting the dashboard**: Launch command and configuration
- **Key features**: Model loading, predictions, visualizations
- **API endpoints**: REST API for programmatic access
- **Visualizations**: Heatmaps, confusion matrices, confidence plots

### Dashboard URL

Once running, access the dashboard at:

```
http://localhost:5000
```

### Key Commands

```bash
# Start dashboard
PYTHONPATH=src .venv/bin/python flask_dashboard.py

# Check if running
curl http://localhost:5000/api/models

# Stop dashboard
Ctrl+C in terminal
```

---

**Navigation:**
← [Previous: 05_api_usage](05_api_usage.ipynb) |
[Next: 07_hyperparameter_sweeps](07_hyperparameter_sweeps.ipynb) →

**Related:** [08_model_evaluation](08_model_evaluation.ipynb) | [10_visualization_experiments](10_visualization_experiments.ipynb)